In [4]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

やりたいこと<br>
・Samedeviceマッチングによって、smn_uidが不明なadvertising_idにsmn_uidを紐づけし、アプリ配信量を増やす<br>

2019年6月5にもらったDrawbridgeリストでidfaを格納したリスト

In [ ]:
db_idfa_table = "segment.drawbridge_idfas" 
print(db_idfa_table)

2019年6月5日にもらったDrawbridgeリストでcookie_idを格納したリスト

In [ ]:
db_cookie_table = "segment.drawbridge_smn_uids" 
print(db_cookie_table)

In [ ]:
query = ( 
    f"""
    select
    count(smn_uid) as uu_smn_uid
    from 
    {db_cookie_table}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_uid = ir.sql_to_pandas(query)

In [ ]:
df_uid.head()/1000000 #1.7億レコード

In [ ]:
query = ( 
    f"""
    select
    count(advertising_id) as uu_ad_id
    from 
    {db_idfa_table}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_idfa = ir.sql_to_pandas(query)

In [ ]:
df_idfa.head()/1000000 #4500万レコード

In [ ]:
query = ( 
    f"""
    select
    count(distinct t1.drawbridge_consumer_id) as cnt_db_id
    from 
    {db_idfa_table} t1
    inner join 
    {db_cookie_table} t2
    on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
    where 
    smn_uid > 0 
    and advertising_id is not null
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_tmp = ir.sql_to_pandas(query)

In [ ]:
df_tmp.head()/1000000

↑smn_uidとadvertising_id両方を持っているユーザーは約1300万ユーザー

In [ ]:
#両IDを持っている（セイムデバイスの可能性のある）Drawbridge_idリストを生成
db_list = f"dm_tmp.samedevice_userlist_190723"
print(db_list)

In [ ]:
query = ( 
    f"""
    select
    distinct t1.drawbridge_consumer_id
    from 
    {db_idfa_table} t1
    inner join 
    {db_cookie_table} t2
    on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
    where 
    smn_uid > 0 
    and advertising_id is not null
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_db = ir.sql_to_pandas(query)
        ir.upload_dataframe(db_list, df_db, ['string'])

紐づけが可能そうなadvertising_idリストを作成する

In [ ]:
adid_list = f"segment.ya_samedevice_adid_userlist_190723"
print(adid_list)

ファイルが重くなるので、分割して実施して作成

In [ ]:
%%slack_notify @yusuke_akada
temp_table = "dm_tmp.ya_temp_samedevice_190723"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {adid_list}
            (
            drawbridge_consumer_id string,
            advertising_id string,
            type string
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    

In [ ]:
# テキストファイル単体ごとに読込、DataFrame化
for i in range(0,200):
    # 親テーブルにInsert
    query = (
        f"""
        insert overwrite table {adid_list}
        (
        drawbridge_consumer_id,
        advertising_id,
        type
        )

        partition
        (
        block = {i}
        )
        
        select
        drawbridge_consumer_id,
        advertising_id,
        type
        from 
        segment.drawbridge_idfas
        where
        block = {i} 
        and drawbridge_consumer_id in (select drawbridge_consumer_id from {db_list})
                
        """
        )
    ir.execute_query_no_ret(query) 

In [ ]:
query = ( 
    f"""
    select
    count(advertising_id) as ad_id
    from 
    {adid_list}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_adid = ir.sql_to_pandas(query)

In [ ]:
df_adid/1000000

In [ ]:
#↑セイムデバイスの可能性のあるadvertising_id = 36852179

紐づけが可能そうなsmn_uidリストを作成する

In [ ]:
uid_list = f"segment.ya_samedevice_smn_uid_userlist_190723"
print(uid_list)

ファイルが重くなるので、分割して実施する

In [ ]:
%%slack_notify @yusuke_akada
temp_table = "dm_tmp.ya_temp_samedevice_190723"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {uid_list}
            (
            drawbridge_consumer_id string,
            smn_uid bigint,
            type string
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    

In [ ]:
%%slack_notify @yusuke_akada
# テキストファイル単体ごとに読込、DataFrame化
for i in range(0,200):
    # 親テーブルにInsert
    query = (
        f"""
        insert overwrite table {uid_list}
        (
        drawbridge_consumer_id,
        smn_uid,
        type
        )

        partition
        (
        block = {i}
        )
        
        select
        drawbridge_consumer_id,
        smn_uid,
        type
        from 
        segment.drawbridge_smn_uids
        where
        block = {i} 
        and drawbridge_consumer_id in (select drawbridge_consumer_id from {db_list})
                
        """
        )
    ir.execute_query_no_ret(query) 

In [ ]:
query = ( 
    f"""
    select
    count(smn_uid) as uid
    from 
    {uid_list}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_uid = ir.sql_to_pandas(query)

In [ ]:
df_uid/1000000

In [ ]:
#↑セイムデバイスの可能性のあるsmn_uid = 117692826

### idfa_idベース/Cookieでbidresponseを参照

### 統合版

In [ ]:
%%slack_notify @yusuke_akada

seed_adid_table = 'segment.ya_samedevice_adid_userlist_190723' #紐づけが可能そうなadidを持っているDrawbridge_consumer_idリスト
seed_smnuid_table = 'segment.ya_samedevice_smn_uid_userlist_190723' #紐づけが可能そうなsmn_uidを持っているDrawbridge_consumer_idリスト

idfa_req_table = "segment.ya_db_req_idfa_190726"   # idfaリストの最終的な対応table

smnuid_req_table = "segment.ya_db_req_smnuid_190726"   # idfaリストの最終的な対応table

# 親テーブルにInsert
for i in range(0,200):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        for d in range(1,2):
            query = (
                    f"""
                    insert overwrite table {idfa_req_table}
                    (
                    advertising_id
                    ,smn_uid
                    ,drawbridge_consumer_id
                    ,user_agent_os_family
                    ,user_agent_device_family
                    ,user_agent_browser_family
                    ,user_agent_os_major
                    ,user_agent_os_minor
                    ,remote_address
                    )
                    partition
                    (
                    yymmdd = to_date(days_add(now(),-{d}))
                    ,block = {i}
                    )
                    select
                    distinct t1.advertising_id
                    ,t1.smn_uid
                    ,drawbridge_consumer_id
                    ,user_agent_os_family
                    ,user_agent_device_family
                    ,user_agent_browser_family
                    ,user_agent_os_major
                    ,user_agent_os_minor
                    ,remote_address
                    from
                    dsplog.bid_response t1
                    inner join 
                    {seed_adid_table} t2
                    on t1.advertising_id = t2.advertising_id
                    where 
                    concat_ws('-',year,month,day) between to_date(days_add(now(),-{d})) and to_date(days_add(now(),-{d})) 
                    and user_agent_os_family in('Android','iOS')
                    and is_app = True
                    and t1.advertising_id is not null
                    and t1.smn_uid is null
                    and block = {i}
                    """
                )
            ir.execute_query_no_ret(query) 
            
            query = (
                f"""
                insert overwrite table {smnuid_req_table}
                (
                smn_uid,
                advertising_id,
                drawbridge_consumer_id,
                user_agent_os_family,
                user_agent_device_family,
                user_agent_browser_family,
                user_agent_os_major,
                user_agent_os_minor,
                remote_address
                )
                partition
                (
                yymmdd = to_date(days_add(now(),-{d}))
                ,block = {i}
                )
                select
                distinct t1.smn_uid
                ,t1.advertising_id
                ,drawbridge_consumer_id
                ,user_agent_os_family
                ,user_agent_device_family
                ,user_agent_browser_family
                ,user_agent_os_major
                ,user_agent_os_minor
                ,remote_address
                from
                dsplog.bid_response t1
                inner join 
                {seed_smnuid_table} t2
                on t1.smn_uid = t2.smn_uid
                where 
                concat_ws('-',year,month,day) between to_date(days_add(now(),-{d})) and to_date(days_add(now(),-{d})) 
                and user_agent_os_family in('Android','iOS')
                and t1.smn_uid is not null
                and t1.advertising_id is null
                and block = {i}
                """
            )
            ir.execute_query_no_ret(query) 

### セイムデバイス済みテーブルを作成する

In [ ]:
uid_table = "segment.samedevice_user_list_190726"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_samedevice"    # 分割処理用のtemporary table 実行後drop

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            advertising_id string
            ,smn_uid bigint
            ,drawbridge_consumer_id string
            ,user_agent_os_family string
            ,user_agent_device_family string
            ,user_agent_browser_family string
            ,user_agent_os_major string
            ,user_agent_os_minor string
            ,remote_address string
            )
            
            partitioned by 
            (
            yymmdd string
            ,block int

            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [ ]:
%%slack_notify @yusuke_akada

sd_original_table = "segment.samedevice_user_list_190726"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_samedevice"    # 分割処理用のtemporary table 実行後drop

idfa_table = "segment.ya_db_req_idfa_190726"
smnuid_table = "segment.ya_db_req_smnuid_190726"

# 親テーブルにInsert
for yymmdd in ['2019-09-01','2019-09-02']:
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        for i in range(0,200):
            query = (
                f"""
                insert overwrite table {sd_original_table}
                (
                advertising_id,
                smn_uid,
                drawbridge_consumer_id,
                user_agent_os_family
                ,user_agent_device_family
                ,user_agent_browser_family
                ,user_agent_os_major
                ,user_agent_os_minor
                ,remote_address
                )
                partition
                (
                yymmdd = '{yymmdd}'
                ,block = {i}
                )
                select
                distinct t1.advertising_id
                ,t2.smn_uid
                ,t1.drawbridge_consumer_id
                ,t1.user_agent_os_family
                ,t1.user_agent_device_family
                ,t1.user_agent_browser_family
                ,t1.user_agent_os_major
                ,t1.user_agent_os_minor
                ,t1.remote_address
                from 
                {idfa_table} t1
                inner join 
                {smnuid_table} t2
                on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
                and t1.user_agent_os_family = t2.user_agent_os_family
                and t1.user_agent_device_family = t2.user_agent_device_family
                and t1.user_agent_os_major = t2.user_agent_os_major
                and t1.user_agent_os_minor = t2.user_agent_os_minor
                and t1.remote_address = t2.remote_address
                and t1.yymmdd = t2.yymmdd
                and t1.block = {i}
                and t2.block = {i}
                """
            )
            ir.execute_query_no_ret(query) 

In [16]:
sd_original_table = "segment.samedevice_user_list_190726" 
print(sd_original_table)

segment.samedevice_user_list_190726


#advertising_id_typeがないので付与する

In [13]:
samedevice_fix_table = "segment.samedevice_user_list_fix_1909"   # idfaリストの最終的な対応table
print(samedevice_fix_table)

segment.samedevice_user_list_fix_1909


In [14]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {samedevice_fix_table}
            (
            advertising_id string
            ,smn_uid bigint
            ,drawbridge_consumer_id string
            ,yymmdd string
            ,advertising_id_type bigint
            )
            
            partitioned by 
            (
            block_no int

            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [17]:
start_date = '2019-08-01'
end_date = '2019-08-31'

In [ ]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert
for i in range(0,200):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

        query = (
            f"""
            insert overwrite table {samedevice_fix_table}
            (
            advertising_id,
            smn_uid,
            drawbridge_consumer_id,
            yymmdd,
            advertising_id_type                
            )
            partition
            (
            block_no = {i}
            )
            select
            t1.advertising_id
            ,t1.smn_uid
            ,t1.drawbridge_consumer_id
            ,yymmdd              
            ,case when type in ('android_idfa') then 1 
            when type in ('ios_idfa') then 2 
            else 0 end as advertising_id_type
            from 
            {sd_original_table} t1
            left join
            segment.drawbridge_idfas t2
            on t1.advertising_id = t2.advertising_id
            where t1.block = {i}
            and t2.block = {i}
            and t1.smn_uid not in (select smn_uid from dm.idfa_list_nn_pq where concat_ws('-',year,month,day) between '{start_date}' and '{end_date}')
            and t1.advertising_id not in (select advertising_id from dm.idfa_list_nn_pq where concat_ws('-',year,month,day) between '{start_date}' and '{end_date}')
            """
        )
        ir.execute_query_no_ret(query) 

In [27]:
query = (
        f"""
        select *
        from {samedevice_fix_table}
        limit 10
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_list = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [ ]:
df_list.head()

In [28]:
#100%リスト

query = (
        f"""
        select
        distinct smn_uid
        ,advertising_id
        ,advertising_id_type
        from {samedevice_fix_table}
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_all = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [29]:
df_all.shape[0]

2476076

In [30]:
df_all.head()

,smn_uid,advertising_id,advertising_id_type
0,936000177272917,364C92C54C194AC9B7C835A6539AB656,2
1,1661000006917040,EAA3DA89C6344E19AB07B3F9E70BF88C,1
2,895000029460529,DCF79B47F6C44946BEB56659C634821A,2
3,1764000002212651,291F8DC7FA68497F9D5BAF6D057C113C,1
4,1722000004207405,4A5EDF9858D64B63929E8D4BE862712E,1


In [31]:
df_dup = df_all.drop_duplicates(subset='advertising_id')

In [60]:
df_dup.shape[0]

1935380

In [33]:
df_dup = df_dup.drop_duplicates(subset='smn_uid')

In [34]:
df_dup.shape[0]

1935380

In [64]:
df_dup = df_dup.ix[:,['advertising_id','advertising_id_type','smn_uid']]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [44]:
df_dup.advertising_id_type[df_dup.advertising_id_type == 1] = 91

In [45]:
df_dup.advertising_id_type[df_dup.advertising_id_type == 2] = 92

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
df_dup.head()

,advertising_id,advertising_id_type,smn_uid
0,364C92C54C194AC9B7C835A6539AB656,92,936000177272917
1,EAA3DA89C6344E19AB07B3F9E70BF88C,91,1661000006917040
2,DCF79B47F6C44946BEB56659C634821A,92,895000029460529
3,291F8DC7FA68497F9D5BAF6D057C113C,91,1764000002212651
4,4A5EDF9858D64B63929E8D4BE862712E,91,1722000004207405


In [66]:
df_dup.to_csv('drawbridge_samedevice_list_190904.tsv', sep='\t', index=False)

第一弾のテーブル化

In [54]:
#リスト読み込み
df_user = pd.read_csv("/home/yusuke_akada/notebook/Samedevice/drawbridge_samedevice_list_190731.tsv",sep='\t')
df_user.head()

,advertising_id,advertising_id_type,smn_uid
0,F15B161D5AF84644BACA85BD9C6D2DA6,92,872000011439179
1,CAFE977686C94D64B39537DAE81AC296,92,1129000065152494
2,95AAFF69C69B45D583EBCEA1DCAD07BF,92,895000018861205
3,CF8F5D9B48574388AA0B06326A7EB13C,92,335000048656823
4,34B4F91132334236AC3DB894C437840A,92,2202000103904352


In [55]:
uid_table = f'segment.drawbridge_samedevice_list_1st_20190816'
print(uid_table)

segment.drawbridge_samedevice_list_1st_20190816


In [56]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        ir.upload_dataframe(uid_table, df_user, ['string','bigint','bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Create empty 

第2弾のテーブル化

In [67]:
#リスト読み込み
df_user = pd.read_csv("/home/yusuke_akada/notebook/Samedevice/drawbridge_samedevice_list_190904.tsv",sep='\t')
df_user.head()

,advertising_id,advertising_id_type,smn_uid
0,364C92C54C194AC9B7C835A6539AB656,92,936000177272917
1,EAA3DA89C6344E19AB07B3F9E70BF88C,91,1661000006917040
2,DCF79B47F6C44946BEB56659C634821A,92,895000029460529
3,291F8DC7FA68497F9D5BAF6D057C113C,91,1764000002212651
4,4A5EDF9858D64B63929E8D4BE862712E,91,1722000004207405


In [68]:
uid_table = f'segment.drawbridge_samedevice_list_2nd_20190904'
print(uid_table)

segment.drawbridge_samedevice_list_2nd_20190904


In [69]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        ir.upload_dataframe(uid_table, df_user, ['string','bigint','bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Create empty 

第一弾と統合して1つのセグメントファイルにする

In [2]:
samedevice_fix_table = "segment.samedevice_user_list_by_drawbridge"   # 最終的な対応table
print(samedevice_fix_table)

segment.samedevice_user_list_by_drawbridge


In [5]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {samedevice_fix_table}
            (
            advertising_id string
            ,advertising_id_type bigint
            ,smn_uid bigint
            )
           
            partitioned by 
            (
            create_date string

            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [6]:
insert_table = 'dm_tmp.drawbridge_samedevice_list_1st_20190816'
print(insert_table)

dm_tmp.drawbridge_samedevice_list_1st_20190816


In [7]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

date = '2019-08-10'

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table {samedevice_fix_table}
        (
        advertising_id
        ,advertising_id_type
        ,smn_uid
        )
        partition
        (
        create_date = '{date}'
        )
        select *
        from 
        {insert_table}
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

第2弾と統合して1つのセグメントファイルにする

In [2]:
samedevice_fix_table = "segment.samedevice_user_list_by_drawbridge"   # 最終的な対応table
print(samedevice_fix_table)

segment.samedevice_user_list_by_drawbridge


In [3]:
insert_table = 'segment.drawbridge_samedevice_list_2nd_20190904'
print(insert_table)

segment.drawbridge_samedevice_list_2nd_20190904


In [4]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

date = '2019-09-14'

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table {samedevice_fix_table}
        (
        advertising_id
        ,advertising_id_type
        ,smn_uid
        )
        partition
        (
        create_date = '{date}'
        )
        select *
        from 
        {insert_table}
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     